In [151]:
import math
import time
import numpy as np
import pandas as pd
from hdrh.histogram import HdrHistogram
import pyarrow as pa
import pyarrow.csv as csv
import pyarrow.compute as pc

In [2]:
df = pd.read_csv("./data/processed_full.csv")
df

,gq,age_group,married,household_size,race_recoded,latino_race,latino_origin,latino_origin_detailed,veteran,citizenship,...,eligible_voters,YEAR,STATEFIP,SEX,AGE,HHWT,PERWT,county_id,hhincome_filter,cash_child
0,0,3,2.0,2.0,2,NaN,NaN,NaN,0.0,1,...,1,2020,1,2,84,21,21,1043.0,1.0,0
1,0,2,2.0,NaN,2,NaN,NaN,NaN,0.0,1,...,1,2020,1,2,35,4,2,1039.0,3.0,0
2,0,3,2.0,2.0,4,NaN,NaN,NaN,1.0,1,...,1,2020,1,1,67,6,6,1113.0,3.0,0
3,0,1,NaN,NaN,2,NaN,NaN,NaN,NaN,1,...,0,2020,1,1,6,36,36,1073.0,3.0,0
4,0,1,NaN,NaN,2,NaN,NaN,NaN,NaN,1,...,0,2020,1,1,4,13,13,1031.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15441668,0,2,2.0,NaN,2,NaN,NaN,NaN,0.0,1,...,1,2020,56,2,28,16,10,56005.0,3.0,0
15441669,0,2,3.0,1.0,2,NaN,NaN,NaN,0.0,1,...,1,2020,56,1,58,9,9,56013.0,1.0,0
15441670,0,3,2.0,NaN,2,NaN,NaN,NaN,0.0,1,...,1,2020,56,2,90,3,2,56001.0,3.0,0
15441671,0,1,NaN,NaN,1,5.0,1.0,1.0,NaN,1,...,0,2020,56,1,1,87,237,56005.0,2.0,0


In [4]:
df.dtypes

gq                   int64
age_group            int64
married            float64
household_size     float64
race_recoded         int64
                    ...   
HHWT                 int64
PERWT                int64
county_id          float64
hhincome_filter    float64
cash_child           int64
Length: 76, dtype: object

In [152]:
df_arrow = csv.read_csv("./data/processed_full.csv")
df_arrow

pyarrow.Table
gq: int64
age_group: int64
married: int64
household_size: int64
race_recoded: int64
latino_race: int64
latino_origin: int64
latino_origin_detailed: int64
veteran: int64
citizenship: int64
yrsusa2: int64
yrsusa1: int64
family_structure: int64
three_gen: int64
live_grandkid: int64
responsible_grandkid: int64
kids_gq: int64
preschool: int64
not_enrolled_es: int64
not_enrolled_hs: int64
disconnect_youth: int64
eduatt: int64
eduatt_condensed: int64
lep: int64
spanish: int64
spanish_lep: int64
otherlang: int64
otherlang_lep: int64
lingisol: int64
labforce: int64
employ: int64
selfemp: int64
occ_group: int64
ind_group: int64
wage: int64
wage_15: int64
hhincome: int64
poverty: int64
poverty_child: int64
poverty_employed: int64
free_lunch: int64
foodstamp: int64
foodstamp_child: int64
cash: int64
ownership: int64
ownership_child: int64
hvalue: int64
owner_hcost: int64
owner_hcost_child: int64
renter_hcost: int64
renter_hcost_child: int64
overcrowd: int64
overcrowd_child: int64
int

In [149]:
def weighted_median_grouped(values, weights, group):
    quantiles = pd.DataFrame({"values": values, "weights": weights, "group": group}).groupby(
        ["group", "values"]).sum().groupby("group").cumsum().groupby("group")["weights"].transform(lambda x: x / x.max())
    return quantiles[quantiles >= 0.5].reset_index().groupby("group").first()["values"]

In [200]:
weighted_median_grouped(df.household_size, df.HHWT, df.STATEFIP)

group
1     2.0
2     2.0
4     2.0
5     2.0
6     2.0
8     2.0
9     2.0
10    2.0
11    2.0
12    2.0
13    2.0
15    2.0
16    2.0
17    2.0
18    2.0
19    2.0
20    2.0
21    2.0
22    2.0
23    2.0
24    2.0
25    2.0
26    2.0
27    2.0
28    2.0
29    2.0
30    2.0
31    2.0
32    2.0
33    2.0
34    2.0
35    2.0
36    2.0
37    2.0
38    2.0
39    2.0
40    2.0
41    2.0
42    2.0
44    2.0
45    2.0
46    2.0
47    2.0
48    2.0
49    2.0
50    2.0
51    2.0
53    2.0
54    2.0
55    2.0
56    2.0
Name: values, dtype: float64

In [4]:
def weighted_median(values, weights):
    dict = {}
    total = 0
    running_total = 0

    for i in range(len(values)):
        try:
            f = int(values[i])
            w = int(weights[i])
        except:
            continue

        if f in dict:
            dict[f] += w
        else:
            dict[f] = w
        total += w

    for i in sorted(dict):
        running_total += dict[i]
        if running_total >= total / 2.0:
            return i

    return -1

In [9]:
weighted_median(df.hhincome, df.HHWT)

64000

In [5]:
def weighted_median_histogram(values, weights, min=1, max=99, precision=5):
    histogram = HdrHistogram(min, max, precision)

    for i in range(len(values)):
        try:
            f = int(values[i])
            w = int(weights[i])
        except:
            continue

        histogram.record_value(f, w)

    return histogram.get_value_at_percentile(50.0)

In [255]:
def weighted_median_histogram_np(values, weights, min=1, max=99, precision=5):
    histogram = HdrHistogram(min, max, precision)
    df = pd.DataFrame({"values": values, "weights": weights})
    df[df.notnull().all(1)].apply(lambda x: histogram.record_value(x['values'], int(x['weights'])), axis=1)

    return histogram.get_value_at_percentile(50.0)

In [256]:
weighted_median_histogram_np(df.hhincome, df.HHWT)

60749

In [153]:
def weighted_median_arrow(values, weights):
    table = pa.Table.from_arrays(
        [values, weights], names=['values', 'weights'])
    agg = table.group_by('values').aggregate(
        [('weights', 'sum')]).filter(pc.field("values").is_valid())
    aggSorted = agg.sort_by([('values', 'ascending')])
    medianIndex = pc.divide(pc.multiply(pc.cumulative_sum(
        aggSorted["weights_sum"]), 10), (pc.sum(aggSorted["weights_sum"]))).index(5)
    return aggSorted["values"][medianIndex.as_py()].as_py()

In [197]:
start = time.time()
weighted_median_arrow(df_arrow['hhincome'], df_arrow['HHWT'])
end = time.time()
print(end - start)

0.06273126602172852


In [72]:
df.groupby('STATEFIP').apply(
    lambda x: weighted_median_arrow(x.hhincome, x.HHWT))

STATEFIP
1     51098.0
2     78346.0
4     60819.0
5     48574.0
6     77312.0
8     74428.0
9     78880.0
10    68039.0
11    90000.0
12    56800.0
13    60400.0
15    81837.0
16    58448.0
17    67330.0
18    57509.0
19    61154.0
20    60406.0
21    51500.0
22    49900.0
23    58000.0
24    86074.0
25    83057.0
26    58606.0
27    72260.0
28    45562.0
29    56696.0
30    55990.0
31    62468.0
32    61458.0
33    76949.0
34    84390.0
35    50201.0
36    70096.0
37    55687.0
38    64000.0
39    57500.0
40    53431.0
41    65004.0
42    62875.0
44    69900.0
45    53912.0
46    59327.0
47    54000.0
48    63000.0
49    73300.0
50    61356.0
51    75498.0
53    76154.0
54    47283.0
55    62880.0
56    64000.0
dtype: float64

In [13]:
values = df.hhincome
weights = df.HHWT
histogram = HdrHistogram(1, 99, 5)
for i in range(len(values)):
    try:
        f = int(values[i])
        w = int(weights[i])
    except:
        continue

    histogram.record_value(f, w)

In [75]:
weighted_median_arrow(df_arrow['hhincome'], df_arrow['HHWT'])

64000

In [ ]:
histogram = HdrHistogram(1, 99, 5)
df.apply(lambda x:  histogram.record_value(f, w) weighted_median_arrow(x.hhincome, x.HHWT))